# Web scrapping del diario La Nacion
## Solo titulares entre fechas predeterminadas

In [63]:
# Importamos todo lo necesario
# -- coding: utf-8 --
import requests
from bs4 import BeautifulSoup
import pickle
from time import sleep
import pandas as pd 
import re
import string
from collections import Counter
import nltk
from nltk.corpus import PlaintextCorpusReader
from sklearn.feature_extraction import text 
from wordcloud import WordCloud
import matplotlib.pyplot as plt
import numpy as np
from textblob import TextBlob
import math
import locale
from gensim import matutils, models
import scipy.sparse
from nltk import word_tokenize, pos_tag
from sklearn.feature_extraction.text import CountVectorizer
import csv
import datetime
# debbuging...
import pdb
from IPython.core.debugger import set_trace


In [68]:
# Fechas que vamos a analizar
# 2006 09 10
# 2016 02 18

# Seteamos el LOCALE a español (por las fechas...)
locale.setlocale(locale.LC_TIME, "es_AR.utf8")

# El buscador de lanacion pagina de a 10 los resultados
# En el url se colocan las fechas de inicio y final
# y las ahcemos coincidir con las que tenemos de perfil
# por ultimo se ordena de viejoa nuevo y se elige la page del paginado
# segun el indice, hay 477 paginas
url_page = "https://buscar.lanacion.com.ar/ediciones anteriores/date-20060910,20160218/sort-old/page-"


In [69]:
def add_to_csv(fila):
    """ Recibe la fila y lo graba en el archivo tit_perfil.csv
    """
    with open('tit_lanacion.csv', 'a') as csvFile:
        writer = csv.writer(csvFile)
        writer.writerow(fila)
    csvFile.close()

In [70]:
for pagina in range(1,478):
#for pagina in range(1,3):
    # cargar la pagina
    # despejar todos los li.floatFix  
    # recorrerlos
    # despejar el spam class="tituloNota" h2 que es el titular
    # luego el p que contiene la entradilla
    # Del div class="data" sacamos la fecha en formato 16 OCT 2005
    # y del primer a de ese div obtenemos la seccion del articulo
        url = url_page + str(pagina)
        #print(url)
        req = requests.get(url)
        # Comprobamos que la petición nos devuelve un Status Code = 200
        status_code = req.status_code
        if status_code == 200:
            soup = BeautifulSoup(req.content, "lxml")
            for li in soup.find_all('li', attrs={'class': 'floatFix'}):
                #print(li)
                titulo = li.find('span', {'class': 'tituloNota'}).h2.getText().lower()
                #print(titulo)
                fecha_txt = li.find('div',{'class': 'data'}).getText()[:11]
                fecha_object = datetime.datetime.strptime(fecha_txt, '%d %b %Y').date()
                anno =fecha_object.year
                mes = fecha_object.month
                dia = fecha_object.day
                seccion = li.find('div',{'class': 'data'}).a.getText().lower()
                #print(seccion)
                entradilla = li.find('p').getText().lower()
                #print(entradilla)
                fila = [anno, mes, dia, seccion, titulo, entradilla]
                add_to_csv(fila)
                sleep(0.1)

In [61]:
fecha_object = datetime.datetime.strptime('01 FEB 2006', '%d %b %Y').date()